In [21]:
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd
import io
import lxml
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

BASE_URL = "http://www.marathonguide.com/results/browse.cfm?"
YEAR_URL = BASE_URL + "Year=%s" #where %s is a placeholder for year
RESULT_STR = "&Gen=B&Begin=%d&End=%d&Max=%d"
DRIVER = "chromedriver.exe"

driver = webdriver.Chrome(DRIVER)
driver.set_window_size(1920,1080)

#scrape race list page
def return_race_list(year):
    r = requests.get(YEAR_URL%"2016")
    c = r.content
    soup = BeautifulSoup(c, 'html.parser')
    table = soup.find_all('table')[4].find_all('a')
    race_count = int(len(table)/2)
    raw_list = table[:race_count]
    race_list = [name.text for name in raw_list]
    link_list = [re.findall(r'href="(.*)"',str(i))[0] for i in raw_list]
    clean_race_dict = dict(zip(race_list, link_list))
    return clean_race_dict

#scrape race page
def return_option_list(race_id):
#     str_chop_len = len("browse.cfm?")
#     race_name = list(clean_race_dict.keys())[0]
#     race_id = clean_race_dict[race_name][str_chop_len:]
    get_race_page = requests.get(BASE_URL + "MIDD=15170417")
    content_race_page = get_race_page.content
    soup_race_page = BeautifulSoup(content_race_page, 'html.parser')
    race_info = soup_race_page.find_all('table')[1].find_all('b')
    location = race_info[-3].text
    date = race_info[-2].text
#    race_meta = {'race': race_name, 'date': date, 'location': location}
    option_list = [option.text for option in soup_race_page.find_all('option')][1:]
    last_overall_index = option_list.index("Men's Results")
    option_list[1:last_overall_index][0].split(" - ")
    finisher_count = int(option_list[1:last_overall_index][-1].split(" - ")[1])
    return option_list

#TODO:
#init global df
#iterate through last 10 years to scrape race lists for IDs
    #init race year df
    #iterate though List IDs to generate race pages
        #init race results
        #iterate through option list to generate results
            #append results to race list
#output df to .csv
option_list = return_option_list("MIDD=15170417")
url = BASE_URL + "MIDD=15170417"
driver.get(url)
#select dropdown to option list index 1
select_fr = Select(driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[1]/td[2]/table[3]/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td[1]/form/select'))
select_fr.select_by_visible_text(option_list[1])

#find view button and click
view = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[1]/td[2]/table[3]/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td[1]/form/p[3]/input[3]')
view.click()

#return page source from driver and parse with BeatifulSoup
time.wait(4)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

In [29]:
#find all relevant tags in the table body and cough clean
#raw_results = soup.find_all('tbody')[13].text.split('\n\n\n')[5:-1]

#go back to option selection page to scrape new results
#driver.back()

df = pd.DataFrame()

df = df.append([i.split('\n') for i in raw_results[1:]])

df.columns = ["Name(age)", "place", "time", "city,state,country", "AG.AdjTime", "BQ"]

# cleaning dataframe with inplace functions
df['DivPl'] = df['place'].map(lambda x: x.split('/')[1].split()[0])
df['Div'] = df['place'].map(lambda x: x.split()[-1])
df['place'] = df['place'].map(lambda x: x.split()[0])
df['city'] = df['city,state,country'].map(lambda x: x.split(',')[0])
df['state'] = df['city,state,country'].map(lambda x: x.split(',')[1])
df['country'] = df['city,state,country'].map(lambda x: x.split(',')[2])
df.drop('city,state,country', axis = 1, inplace = True)

ValueError: Length mismatch: Expected axis has 4 elements, new values have 6 elements

In [30]:
df

,0,1,2,3
0,Chris C. Stone (M36),"2:35:26 101 89 / 76 M18-39 Edmonton, AB, CAN",2:35:16,BQ
1,Ryan Cox (M26),"2:35:40 102 90 / 77 M18-39 Anchorage, AK, USA",2:35:40,BQ
2,Ian Nurse (M39),"2:35:41 103 91 / 78 M18-39 Brookline, MA, USA",2:33:11,BQ
3,Scott E Byers (M36),"2:35:44 104 92 / 79 M18-39 Miami, FL, USA",2:35:34,BQ
4,Jacob D Kirk (M24),"2:35:49 105 93 / 80 M18-39 Anchorage, AK, USA",2:35:49,BQ
...,...,...,...,...
95,Scott G. Cichon (M31),"2:40:33 196 174 / 150 M18-39 Pittsburg, KS, USA",2:40:33,BQ
96,Stephen A Eles (M37),"2:40:36 197 175 / 151 M18-39 Houghton, MI, USA",2:39:57,BQ
97,Jim Johnson (M39),"2:40:37 198 176 / 152 M18-39 Madison, NH, USA",2:38:02,BQ
98,Sam D Teigen (M42),"2:40:39 199 177 / 25 M40-49 Hawthorne, NJ, USA",2:34:12,BQ


In [24]:
soup

<html><head><script async="" src="http://rules.quantcount.com/rules-p-64MbHu9Hy9z1w.js"></script><script src="https://m.addthis.com/live/red_lojson/300lo.json?si=5e76f48e09bbd752&amp;bkl=0&amp;bl=1&amp;pdt=724&amp;sid=5e76f48e09bbd752&amp;pub=xa-4cd01a594af15d5e&amp;rev=v8.28.3-wp&amp;ln=en&amp;pc=men&amp;cb=0&amp;ab=-&amp;dp=www.marathonguide.com&amp;fp=results%2Fbrowse.cfm%3FRL%3D1%26MIDD%3D15170417%26Gen%3DB%26Begin%3D101%26End%3D200%26Max%3D26411&amp;fr=results%2Fbrowse.cfm%3FMIDD%3D15170417&amp;of=0&amp;pd=0&amp;irt=1&amp;vcl=1&amp;md=2&amp;ct=1&amp;tct=0&amp;abt=0&amp;cdn=0&amp;pi=1&amp;rb=2&amp;gen=100&amp;chr=UTF-8&amp;mk=Boston%20Marathon%2C2017%20boston%20marathon%2Cmarathons%20races%2Cmarathoning%2Crunning%20marathons%2Ccourse%20maps%2Celevation%20charts%2Ctraining%20coaches%2Cresults%20races%2Crecords%20marathon%20guide%20directory&amp;colc=1584854158455&amp;jsl=12320&amp;uvs=5e76f48b0da2264f001&amp;skipb=1&amp;callback=addthis.cbs.jsonp__198802411716097580" type="text/java

In [10]:
list(return_race_list("2017").keys())

['Last Chance Marathon',
 "New Year's Eve Marathon",
 'Run for the Ranch Marathon',
 'Savage Seven Marathon Day 6',
 'Savage Seven Marathon Day 5',
 'Savage Seven Marathon Day 4',
 'Savage Seven Marathon Day 3',
 'Savage Seven Marathon Day 2',
 'Operation Jack Marathon',
 'Savage Seven Marathon Day 1',
 'Hawk Indoor Marathon',
 '3 Bridges Marathon',
 'Otter Creek Trail Marathon',
 'Bryan/College Station Marathon',
 'Dallas Marathon (formerly the Dallas White Rock Marathon)',
 'Honolulu Marathon',
 'Brazos Bend Marathon',
 'Buckeye Marathon/Half Marathon/10K',
 'Hoover Dam Marathon',
 'Kiawah Island Marathon',
 'Rocket City Marathon',
 'Roxbury Marathon',
 'The Buckeye Marathon',
 'Tucson Marathon',
 'California International Marathon',
 'Cayman Islands Marathon',
 "Rock 'n' Roll San Antonio Marathon and 1/2 Marathon",
 'Run Barbados Marathon Weekend',
 'Topo Trail Marathon',
 'Baton Rouge Beach Marathon',
 'Birch Bay Ghost Marathon',
 'Death Valley Trail Marathon',
 'Memphis Marathon',

In [11]:
BASE_URL + "MIDD=15170417"

'http://www.marathonguide.com/results/browse.cfm?MIDD=15170417'

In [31]:
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd
import io
import lxml
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

BASE_URL = "http://www.marathonguide.com/results/browse.cfm?"
YEAR_URL = BASE_URL + "Year=" #where %s is a placeholder for year
RESULT_STR = "&Gen=B&Begin=%d&End=%d&Max=%d"
DRIVER = "chromedriver.exe"

#driver.Options(options = ) #TODO set driveroptions to headless

#scrape race list page
def return_race_list(year):
    r = requests.get(YEAR_URL + year)
    c = r.content
    soup = BeautifulSoup(c, 'html.parser')
    table = soup.find_all('table')[4].find_all('a')
    race_count = int(len(table)/2)
    raw_list = table[:race_count]
    race_list = [name.text for name in raw_list]
    link_list = [re.findall(r'href="(.*)"',str(i))[0] for i in raw_list]
    clean_race_dict = dict(zip(race_list, link_list))
    return clean_race_dict

#scrape race page
def return_option_list(race_id, race_name): #"MIDD=15170417"
    # str_chop_len = len("browse.cfm?")
    # race_name = list(clean_race_dict.keys())[0] #moving from static testing to dynamiic
    # race_id = clean_race_dict[race_name][str_chop_len:]
    get_race_page = requests.get(BASE_URL + race_id)
    content_race_page = get_race_page.content
    soup_race_page = BeautifulSoup(content_race_page, 'html.parser')
    race_info = soup_race_page.find_all('table')[1].find_all('b')
    location = race_info[-3].text
    date = race_info[-2].text
    race_meta = {'race': race_name, 'date': date, 'location': location}
    option_list = [option.text for option in soup_race_page.find_all('option')][1:]
    # below code was written to be able to use BeautifulSoup only to scrape html, but page didnt render with direct request
    # last_overall_index = option_list.index("Men's Results")
    # option_list[1:last_overall_index][0].split(" - ")
    # finisher_count = int(option_list[1:last_overall_index][-1].split(" - ")[1])
    return option_list
        

#TODO:
#init global df
main_df = pd.DataFrame()
#intiate driver
#driver = webdriver.Chrome(DRIVER)
#iterate through last 10 years to scrape race lists for IDs
req_years = [str(i) for i in range(2010, 2020)]
#init race year df
for year in req_years:
    print('Building race list for %s'%(year))
    race_dict = return_race_list(year)

    for race in list(race_dict.keys()):
        print('Building option and context for {%s} {%s}'%(race, year))
        race_id = race_dict[race]
        select_option_list = return_option_list(race_id, race)
        #instantiate holding list with raceID, name, year
        raw_race_result_list = [race_id, race, year]
        headers = 0 #switch to capture table headers once
#       driver.get(BASE_URL + race_id[len("browse.cfm?"):])
        time.sleep(5)

#         #iterate through option values
        for option in select_option_list:
#             #option driver with the URL+race_id
#             #pass option value to driver to select
            print(option)

Building race list for 2010
Building option and context for {Last Chance Marathon} {2010}
Building option and context for {Yukon Do It! Marathon} {2010}
Building option and context for {Operation Jack Marathon} {2010}


KeyboardInterrupt: 

In [32]:
race_dict = return_race_list("2019")

for race in list(race_dict.keys()):
    race_id = race_dict[race]
    select_option_list = return_option_list(race_id, race)
    print(select_option_list)

[]
[]
[]


IndexError: list index out of range

In [38]:
race_dict[list(race_dict.keys())[0]]

'browse.cfm?MIDD=4280191230'

In [40]:
["id"] * 8

['id', 'id', 'id', 'id', 'id', 'id', 'id', 'id']